In [7]:
%pip install scikit-learn

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --- ------------------------------------ 0.8/11.0 MB 26.4 MB/s eta 0:00:01
   ------- -------------------------------- 2.0/11.0 MB 25.3 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/11.0 MB 24.7 MB/s eta 0:00:01
   --------------- ------------------------ 4.3/11.0 MB 25.0 MB/s eta 0:00:01
   ------------------- -------------------- 5.5/11.0 MB 25.0 MB/s eta 0:00:01
   ---------------------- ----------------- 6.2/11.0 MB 24.9 MB/s eta 0:00:01
   --------------------------- ------------ 7.5/11.0 MB 24.1 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/11.0 MB 24.4 MB/s eta 0:00:01
   ----------------------------------- ---- 9.6/11.0 MB 23.6 MB/s eta 0:00:01
   -------------------------------------- - 10.6/11.0 MB 23.4 MB/s eta 0:00:01
   -

In [6]:
# Source: https://pynative.com/python-postgresql-tutorial/
from pathlib import Path
import pandas as pd
import psycopg2
from psycopg2 import Error
from sqlalchemy import create_engine, text

# Here you define the credentials
DATABASE = "group_22_2024" # TO BE REPLACED
USER = "group_22_2024"# TO BE REPLACED
PASSWORD = "8IB1GtKMTx2H" # TO BE REPLACED
HOST = "dbcourse.cs.aalto.fi"

db = None

try:
    connection = psycopg2.connect(
        database = DATABASE,
        user = USER,
        password = PASSWORD,
        host = HOST,
        port = "5432"
    )
    connection.autocommit = True

    # Create a cursor to perform database operations
    cursor = connection.cursor()

    # Connect to db using SQLAlchemy create_engine
    DIALECT = "postgresql+psycopg2://"
    db_uri = "%s:%s@%s/%s" % (USER, PASSWORD, HOST, DATABASE)
    engine = create_engine(DIALECT + db_uri)
    psql_conn = engine.connect()

    query = """
        SELECT 
            R.id AS request_id,
            R.title,
            R.beneficiary_id,
            B.name AS beneficiary_name,
            B.address AS beneficiary_address,
            R.number_of_volunteers,
            R.priority_value,
            R.start_date,
            R.end_date,
            R.register_by_date,
            COUNT(A.id) AS num_applications,
            AVG(V.travel_readiness) AS avg_volunteer_travel_readiness,
            COUNT(DISTINCT RS.skill_name) AS num_required_skills,
            COUNT(DISTINCT RL.city_id) AS num_request_locations,
            COUNT(DISTINCT VS.skill_name) AS num_volunteer_skills,
            COUNT(DISTINCT VA.interest_name) AS num_volunteer_interests,
            COUNT(DISTINCT VR.city_id) AS num_volunteer_ranges
        FROM 
            Request R
        LEFT JOIN 
            Application A ON R.id = A.request_id
        LEFT JOIN 
            Volunteer V ON A.volunteer_id = V.id
        LEFT JOIN 
            Beneficiary B ON R.beneficiary_id = B.id
        LEFT JOIN 
            Request_Skill RS ON R.id = RS.request_id
        LEFT JOIN 
            Request_Location RL ON R.id = RL.request_id
        LEFT JOIN 
            Volunteer_Skill VS ON A.volunteer_id = VS.volunteer_id
        LEFT JOIN 
            Volunteer_Area_Of_Interest VA ON A.volunteer_id = VA.volunteer_id
        LEFT JOIN 
            Volunteer_Range VR ON A.volunteer_id = VR.volunteer_id
        GROUP BY 
            R.id, B.name, B.address;

        """
    df = pd.read_sql(text(query), con=psql_conn)

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


PostgreSQL connection is closed


In [5]:
df

,request_id,title,beneficiary_id,beneficiary_name,beneficiary_address,number_of_volunteers,priority_value,start_date,end_date,register_by_date,num_applications,avg_volunteer_travel_readiness,num_required_skills,num_request_locations,num_volunteer_skills,num_volunteer_interests,num_volunteer_ranges
0,1,WorkInTeam,3,Elderly Care,"Bengalinkatu 069, Rusko",14,1,2024-07-25,2024-07-28,2024-07-18,6048,741.587302,4,4,14,12,7
1,2,WorkWithYoung,9,Local Branch,"Pyörökivenpolku 525, Rusko",25,3,2022-05-31,2022-06-01,2024-06-10,20448,402.471831,9,4,14,12,6
2,3,GuideAndTeach,9,Local Branch,"Pyörökivenpolku 525, Rusko",31,2,2024-07-22,2024-07-27,2024-07-12,12432,844.346847,7,2,14,12,7
3,4,GuideAndTeach,5,Homeless Shelter,"Laajasalonbulevardi 97, Rusko",19,1,2024-09-25,2024-09-25,2024-09-17,7200,495.793333,8,2,14,12,5
4,5,WorkWithElderly,4,Youth Centre,"Oraspolku 4, Rautavaara",23,2,2021-03-01,2021-03-07,2021-02-15,27132,826.018576,7,4,14,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,378,WorkWithYoung,4,Youth Centre,"Oraspolku 4, Rautavaara",11,1,2021-12-29,2021-12-31,2021-12-27,6687,894.985195,3,3,13,12,8
378,379,ImmigrantSupport,3,Elderly Care,"Bengalinkatu 069, Rusko",28,1,2020-07-07,2020-07-07,2020-07-05,41320,818.753146,8,5,14,12,8
379,380,WorkWithYoung,3,Elderly Care,"Bengalinkatu 069, Rusko",18,2,2020-07-11,2020-07-14,2020-07-03,32544,899.655605,6,4,14,12,7
380,381,WorkWithYoung,5,Homeless Shelter,"Laajasalonbulevardi 97, Rusko",20,5,2024-06-24,2024-06-25,2024-06-19,37632,752.965774,7,4,14,12,7


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

data = df

# Data preprocessing
data = data.dropna()

# Convert dates to numerical features (e.g., number of days from a reference date)
data['start_date'] = pd.to_datetime(data['start_date'])
data['end_date'] = pd.to_datetime(data['end_date'])
data['register_by_date'] = pd.to_datetime(data['register_by_date'])
reference_date = pd.to_datetime('2020-01-01')
data['start_days'] = (data['start_date'] - reference_date).dt.days
data['end_days'] = (data['end_date'] - reference_date).dt.days
data['register_by_days'] = (data['register_by_date'] - reference_date).dt.days

# Feature selection
# NB: Uses only numerical features, explore adding text features such as skill name by multi-hot encoding
features = data[['priority_value', 'num_applications', 'avg_volunteer_travel_readiness', 
                 'num_required_skills', 'num_request_locations', 'num_volunteer_skills', 
                 'num_volunteer_interests', 'num_volunteer_ranges', 
                 'start_days', 'end_days', 'register_by_days']]
target = data['number_of_volunteers']


In [11]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Model training
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Model evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 17.741636300965325
R^2 Score: 0.8908763679299807


In [18]:
X_test

,priority_value,num_applications,avg_volunteer_travel_readiness,num_required_skills,num_request_locations,num_volunteer_skills,num_volunteer_interests,num_volunteer_ranges,start_days,end_days,register_by_days
280,1,12600,928.807143,9,2,13,12,8,1786,1787,1776
248,3,2450,371.854694,2,1,14,12,8,1706,1710,1702
218,1,2376,1245.454545,9,2,10,9,7,247,248,235
327,4,3695,783.507442,5,1,14,12,7,12,13,2
56,0,3635,871.116919,0,5,14,12,8,381,383,377
...,...,...,...,...,...,...,...,...,...,...,...
104,1,11676,925.396711,6,2,14,12,8,1658,1659,1655
167,3,2700,523.138889,5,3,11,11,5,984,988,982
63,5,26760,731.659193,10,3,14,12,6,297,298,292
232,0,585,660.470085,1,1,14,12,8,689,691,685


In [23]:
# Usage:
# For a request with the following features,
req = np.array([1, 12600, 928.807143, 9, 2, 13, 12, 8, 1786, 1787, 1776]).reshape(1, -1)
# the model predicts numbers of volunteers as follows.
res = model.predict(req)
print("Number of volunteers needed: ", res[0])

Number of volunteers needed:  32.47812466913521


C:\Users\mahan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
